In [1]:
import copy

In [2]:
import logging

class LoggerMixin(object):
    @property
    def logger(self):
        name = '.'.join([
            self.__module__,
            self.__class__.__name__,
            str(id(self))
        ])
        logger = logging.getLogger(name)
        chandler = logging.StreamHandler()
        chandler.setLevel(logging.DEBUG)
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        chandler.setFormatter(formatter)
        logger.addHandler(chandler)
        logger.setLevel(logging.DEBUG)
        return logger


In [3]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")


DEBUG:root:test


In [4]:
logger.handlers

[<StreamHandler stderr (NOTSET)>]

In [5]:
class Matrix(LoggerMixin, object):
    def __init__(self, data):
        super().__init__()
        n = len(data)
        m = len(data[0])
        assert all(len(sub_array) == m for sub_array in data)
        self.data = [[float(element) for element in row] for row in data]
        self.shape = (n, m)
        self.logger.info("hello!")

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        return "Matrix([\n    " \
            + "\n    ".join(repr(row) for row in self.data) + "\n])"

    def __getitem__(self, item):
        return self.data[item]

    def copy(self):
        return Matrix(copy.deepcopy(self.data))

    def multiply_by(self, other):
        assert other.shape == tuple(reversed(self.shape))
        return Matrix([
            [
                float(sum(
                    self.data[self_row][self_column] \
                        * other.data[other_row][other_column]
                    for self_column, other_row in 
                        zip(range(self.shape[1]), range(other.shape[0]))
                ))
                for other_column in range(other.shape[1])
            ]
            for self_row in range(self.shape[0])
        ])

    @classmethod
    def identity(cls, n):
        matrix = cls([
            [0 for _ in range(n)]
            for _ in range(n)
        ])
        for j in range(n):
            matrix[j][j] = 1.0
        return matrix
        
    def is_square(self):
        return self.shape[0] == self.shape[1]

    def invert(self):
        n = self.shape[0]
        operations_so_far = Matrix.identity(n)
        self_copy = self.copy()

        for k in range(n):
            print(f"processing column {k}")
            print("self_copy =", self_copy)

            # reorder rows
            row_reordering = \
                list(range(k)) + \
                list(reversed(sorted(
                    range(k, n),
                    key=lambda j: abs(self_copy[j][k])
            )))
            reorder_op = Matrix([Matrix.identity(n).data[j] for j in row_reordering])
            self.logger.info("reorder_op =" + str(reorder_op))
            operations_so_far = reorder_op.multiply_by(operations_so_far)
#             print("operations_so_far =", operations_so_far)
            self_copy = reorder_op.multiply_by(self_copy)
            print("self_copy =", self_copy)

            # normalize this row
            normalization_op = Matrix.identity(n)
            normalization_op[k][k] = 1.0 / self_copy[k][k]
            print("normalization_op = ", normalization_op)
            operations_so_far = normalization_op.multiply_by(operations_so_far)
#             print("operations_so_far =", operations_so_far)
            self_copy = normalization_op.multiply_by(self_copy)
            print("self_copy =", self_copy)

            # eliminate the other rows
            eliminations_op = Matrix.identity(n)
            for j in range(n):
                if j == k:
                    continue
                eliminations_op[j][k] = -1.0 * self_copy[j][k]
            print("eliminations_op =", eliminations_op)
            operations_so_far = eliminations_op.multiply_by(operations_so_far)
            print("operations_so_far =", operations_so_far)
            self_copy = eliminations_op.multiply_by(self_copy)
            print("self_copy =", self_copy)

            print("\n")

        return operations_so_far


# Tests

## matrix inversion

In [6]:
X = Matrix([
    [1, 0],
    [0, 1]
])
# X.invert()

2019-10-08 13:00:56,948 - __main__.Matrix.4508182624 - INFO - hello!
INFO:__main__.Matrix.4508182624:hello!


In [ ]:
X = Matrix([
    [3, 1],
    [4, 2]
])
X_inverse = X.invert()
print("X_inverse = ", X_inverse)
X_inverse.multiply_by(X)

In [ ]:
Matrix([
    [1, 0, 0],
    [0, 2, 0],
    [0, 0, 3]
]).invert()

In [ ]:
Matrix([
    [2, 5, 0],
    [1, 0, 1],
    [1, 2, 0],
]).invert()

In [ ]:
Matrix([
    [1, 0],
    [0, 1]
]).invert()

In [ ]:
Matrix([
    [2, 0],
    [0, 1]
]).invert()

In [ ]:
Matrix([
    [1, 0],
    [0, 2]
]).invert()

In [ ]:
Matrix([
    [1, 0, 0],
    [0, 2, 0],
    [0, 0, 1]
]).invert().multiply_by(Matrix([
    [1, 0, 0],
    [0, 2, 0],
    [0, 0, 1]
]))

## matrix multiplication


In [ ]:
Matrix([[0.5]]).multiply_by(Matrix([[2]]))

In [ ]:
Matrix([
    [1, 2],
    [3, 4]
]).multiply_by(Matrix([
    [1, 0],
    [0, 1]
]))

In [ ]:
Matrix([
    [1, 2],
    [3, 4]
]).multiply_by(Matrix([
    [0, 1],
    [1, 0]
]))

In [ ]:
Matrix([
    [1, 2],
    [3, 4]
]).multiply_by(Matrix([
    [2, 0],
    [0, 2]
]))

In [ ]:
Matrix([
    [1, 2],
    [3, 4]
]).multiply_by(Matrix([
    [1, 0],
    [0, 2]
]))

In [ ]:
Matrix([
    [1, 2],
    [3, 4]
]).multiply_by(Matrix([
    [1, 1],
    [0, 1]
]))

### to do - matrix transpose

# Basics

In [ ]:
X = Matrix([[1, 2], [3, 4], [5, 6]])
X

In [ ]:
Y = Matrix([[1, 2, 3], [4, 5, 6]])
Y

In [ ]:
Y[1, 2:3, :]